In [ ]:
""" This project will clean a subdataset of used cars from a 
classifieds section of the German eBay website eBay 
Kleinanzeigen """

In [1]:
import pandas as pd
import numpy as np
autos=pd.read_csv('autos.csv', encoding="Latin-1")

In [ ]:
autos.info()

In [ ]:
autos.head()

In [ ]:
# this dataset has 20 cols and 50,000 observations with 
#'yearOfRegistration', 'powerPS', 'monthOfRegistration', 'nrOfPictures'
# and 'postalCode ' in float64 format, rests are objecgts
# five columns has missing data

In [ ]:
# Cleaning Column Names: converting column names from snakecase, 
# rewording some of the column names to be more descriptive

In [16]:
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_Pictures', 'postal_code',
       'last_seen'],
      dtype='object')

In [3]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_Pictures', 'postal_code',
       'last_seen']
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_Pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [ ]:
# changed col names to snakecase to follow good python naming practice.

In [ ]:
autos.describe(include='all')

In [ ]:
# Any columns that have mostly one value that are candidates to be dropped
# Any columns that need more investigation.
# Any examples of numeric data stored as text that needs to be cleaned.

In [ ]:
autos['price'].head()

In [4]:
autos['price']=autos['price'].str.replace(",", "").str.strip()
autos['price']=autos['price'].str.replace("$", "").astype(float)
autos['odometer']=autos['odometer'].str.replace(",", "").str.strip()
autos['odometer']=autos['odometer'].str.replace("km", "").astype(float)

In [ ]:
autos['price'].head()
autos['odometer'].head()

In [5]:
autos.rename({'odometer':'odometer_km'}, axis=1, inplace=True)

In [ ]:
# explore price and odometer_km columns
print(autos['price'].unique().shape)
print(autos['odometer_km'].unique().shape)

In [ ]:
print(autos['price'].describe())
print(autos['odometer_km'].describe())

In [17]:
#remove rows where price=0
autos=autos[autos['price']!=0]

In [15]:
# extract date from columns: 'date_crawled','ad_created','last_seen'
# missing values are included in the distribution
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index()
autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index()
autos['last_seen'].str[:10].head().value_counts(normalize=True, dropna=False).sort_index()

2016-03-15    0.2
2016-04-01    0.2
2016-04-06    0.6
Name: last_seen, dtype: float64

In [ ]:
# explore the distribution of registration_year.
autos['registration_year'].describe()

In [ ]:
# min year is 1000, max year is 9999, both unreal.  

In [14]:
# count the number of listings with cars that fall outside the 1900 - 2016
# to see if it is safe to drop them
test_outside=~autos["registration_year"].between(1900,2016)
test_outside.value_counts(normalize=True, dropna=False).sort_index()

False    1.0
Name: registration_year, dtype: float64

In [13]:
test_between=autos["registration_year"].between(1900,2016)
test_between.value_counts(normalize=True, dropna=False).sort_index()

True    1.0
Name: registration_year, dtype: float64

In [18]:
# less than 3% cars found outside 1900 - 2016 period, 
# check data betwee 1990 and 2016
sele_condition=autos['registration_year'].between(1900,2016)
autos=autos[sele_condition]

In [19]:
autos['registration_year'].value_counts(normalize=True).sort_index().head()

1910    0.000107
1927    0.000021
1929    0.000021
1931    0.000021
1934    0.000043
Name: registration_year, dtype: float64

In [20]:
# cars before 1988 are less than 2%, remove them next
autos=autos[autos['registration_year'] >= 1988]

In [ ]:
autos['brand'].describe()

In [21]:
# selecting top 10 vehicles
selection=autos['brand'].value_counts().head(10)
print(type(selection))
print(selection)

<class 'pandas.core.series.Series'>
volkswagen       9656
bmw              5081
opel             4946
mercedes_benz    4319
audi             4016
ford             3192
renault          2191
peugeot          1392
fiat             1165
seat              852
Name: brand, dtype: int64


In [22]:
# obtain mean prices for top 10 selling vehicles
brand_mean_prices={}
for i in range(len(selection)):
    if selection.index[i] not in brand_mean_prices:
        avg=autos.loc[autos['brand']==selection.index[i], 'price'].mean()
        brand_mean_prices[selection.index[i]]=avg.round(2)
brand_mean_prices

{'audi': 9375.65,
 'bmw': 8575.78,
 'fiat': 2705.09,
 'ford': 3751.19,
 'mercedes_benz': 31657.73,
 'opel': 5448.93,
 'peugeot': 3088.34,
 'renault': 2425.55,
 'seat': 4402.39,
 'volkswagen': 6527.25}

In [ ]:
# It looks like mercedes_benz is the most expensive vehicle, followed by 
# audi, bmw, volkswagen and opel and seat. renault is the cheapest.  
# It's price is 4 times above average!

In [23]:
top_prices=brand_mean_prices

In [24]:
del top_prices['renault']
del top_prices['peugeot']
del top_prices['fiat']
del top_prices['ford']

In [25]:
# converting top_dict to pandas series
bmp_series = pd.Series(top_prices)
print(bmp_series)

audi              9375.65
bmw               8575.78
mercedes_benz    31657.73
opel              5448.93
seat              4402.39
volkswagen        6527.25
dtype: float64


In [26]:
# obtain mean mileage for top 6 selling vehicles
brand_mean_mileages={}
for i in range(len(bmp_series)):
    if bmp_series.index[i] not in brand_mean_mileages:
        avg=autos.loc[autos['brand']==bmp_series.index[i], 'odometer_km'].mean()
        brand_mean_mileages[bmp_series.index[i]]=avg.round(2)
brand_mean_mileages

{'audi': 129094.87,
 'bmw': 132722.89,
 'mercedes_benz': 131044.22,
 'opel': 129834.21,
 'seat': 121267.61,
 'volkswagen': 129121.79}

In [27]:
# converting brand_mean_mileage dictionary to pandas series 
tm_series = pd.Series(brand_mean_mileages)  
df = pd.DataFrame(bmp_series, columns=['mean_prices'])
df['mean_mileages']=tm_series
df

,mean_prices,mean_mileages
audi,9375.65,129094.87
bmw,8575.78,132722.89
mercedes_benz,31657.73,131044.22
opel,5448.93,129834.21
seat,4402.39,121267.61
volkswagen,6527.25,129121.79


In [ ]:
# price is not correlated with mileage, 
# as all other vehicles also have over 100k mileage as Benz 

In [36]:
autos['gearbox'].value_counts()

manual       34094
automatic     9636
Name: gearbox, dtype: int64

In [32]:
# changing values in gearbox from German to English
autos.loc[autos['gearbox']=='manuell', 'gearbox']='manual'
autos.loc[autos['gearbox']=='automatik', 'gearbox']='automatic'
autos['gearbox'].value_counts()

manual       34094
automatic     9636
Name: gearbox, dtype: int64

In [39]:
# changing dates to more readible format
autos['date_crawled']=autos['date_crawled'].str[:10]
autos['ad_created']=autos['ad_created'].str[:10]
autos['last_seen']=autos['last_seen'].str[:10]
mycopy.head(2)

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_Pictures,postal_code,last_seen
0,2016-03-26,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000.0,control,bus,2004,manual,158,andere,150000.0,3,lpg,peugeot,nein,2016-03-26,0,79588,2016-04-06
1,2016-04-04,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500.0,control,limousine,1997,automatic,286,7er,150000.0,6,benzin,bmw,nein,2016-04-04,0,71034,2016-04-06


In [41]:
autos['date_crawled']=autos['date_crawled'].str.replace("-", "").str.strip()
autos['ad_created']=autos['ad_created'].str.replace("-", "").str.strip()
autos['last_seen']=autos['last_seen'].str.replace("-", "").str.strip()
mycopy.head(2)

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_Pictures,postal_code,last_seen
0,20160326,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000.0,control,bus,2004,manual,158,andere,150000.0,3,lpg,peugeot,nein,20160326,0,79588,20160406
1,20160404,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500.0,control,limousine,1997,automatic,286,7er,150000.0,6,benzin,bmw,nein,20160404,0,71034,20160406
